In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

### Ordinary Least Squares (OLS)

 **Coefficient** and **Intercept**, are the parameters of the fit line. 
Given that it is a multiple linear regression, with 3 parameters, and knowing that the parameters are the intercept and coefficients of hyperplane, sklearn can estimate them from our data. Scikit-learn uses plain Ordinary Least Squares method to solve this problem.

OLS is a method for estimating the unknown parameters in a linear regression model. OLS chooses the parameters of a linear function of a set of explanatory variables by minimizing the sum of the squares of the differences between the target dependent variable and those predicted by the linear function. In other words, it tries to minimizes the sum of squared errors (SSE) or mean squared error (MSE) between the target variable (y) and our predicted output (yhat) over all samples in the dataset.

OLS can find the best parameters using of the following methods:

```
- Solving the model parameters analytically using closed-form equations
- Using an optimization algorithm (Gradient Descent, Stochastic Gradient Descent, Newton’s Method, etc.)
```

In [2]:
df = pd.read_csv('data/fuel_consumption_co2.csv')

In [3]:
columns = [
    'ENGINESIZE',
    'CYLINDERS',
    'FUELCONSUMPTION_CITY',
    'FUELCONSUMPTION_HWY',
    'FUELCONSUMPTION_COMB',
    'CO2EMISSIONS'
]
cdf = df[columns]

In [4]:
# train test split
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

In [5]:
independents = [
    'ENGINESIZE',
    'CYLINDERS',
    'FUELCONSUMPTION_CITY',
    'FUELCONSUMPTION_HWY'
]
dependent = ['CO2EMISSIONS']

In [6]:
regr = LinearRegression()
X_train = np.asanyarray(train[independents])
y_train = np.asanyarray(train[dependent])

In [7]:
regr.fit(X_train, y_train)
print('Coefficients: ', regr.coef_)

Coefficients:  [[11.85159853  6.22788704  6.49985737  3.01636028]]


In [8]:
y_hat = regr.predict(test[independents])
X_test = np.asanyarray(test[independents])
y_test = np.asanyarray(test[dependent])

In [9]:
print('Residual sum of squares: %.2f' % np.mean((y_hat - y_test) ** 2))
print('Variance score: %.2f' % regr.score(X_test, y_test))

Residual sum of squares: 602.09
Variance score: 0.84


### Gradient Descent
When the number of rows in your data set is less than 10,000, you can think of OLS as an option. However, for greater values, you should try other faster approaches. The second option is to use an optimization algorithm to find the best parameters. That is, you can use a process of optimizing the values of the coefficients by iteratively minimizing the error of the model on your training data. For example, you can use gradient descent which starts optimization with random values for each coefficient, then calculates the errors and tries to minimize it through y's changing of the coefficients in multiple iterations. Gradient descent is a proper approach if you have a large data set. Please understand however, that there are other approaches to estimate the parameters of the multiple linear regression that you can explore on your own.

In [10]:
# normalize independent variables
X_norm = X_train.copy()

min_x = np.min(X_train)
max_x = np.max(X_train)

X_norm = (X_train - min_x) / (max_x - min_x)

In [11]:
# normalize dependent variables
y_norm = y_train.copy()

max_y = np.max(y_train)
min_y = np.min(y_train)

y_norm = (y_train - min_y) / (max_y - min_y)

In [12]:
M = len(y_train)

In [13]:
def grad(theta, X, y):
    return 1 / M * np.sum((X.dot(theta) - y) * X, axis=0).reshape(-1, 1)

In [14]:
def cost_func(theta, X, y):
    return np.sum((X.dot(theta) - y) ** 2, axis=0)[0]

In [15]:
def gradient_descent(theta0, X, y, learning_rate=0.5, epochs=1000, TOL=1e-7):
    theta_history = [theta0]
    j_history = [cost_func(theta0, X, y)]
    
    theta_new = theta0 * 10000
    
    for epoch in range(epochs):
        if epoch % 100 == 0:
            print(f'{epoch:5d}\t{j_history[-1]:7.4f}\t')
            
        dj = grad(theta0, X, y)
        j = cost_func(theta0, X, y)
        
        theta_new = theta0 - learning_rate * dj
        theta_history.append(theta_new)
        j_history.append(j)
        
        if np.sum((theta_new - theta0) ** 2) < TOL:
            print('Convergence achieved.')
            break
            
        theta0 = theta_new

    return theta_new, theta_history, j_history

In [16]:
def predict(X, theta):
    X_norm = (X - min_x) / (max_x - min_x)
    y_pred_norm = X_norm.dot(theta)
    
    return y_pred_norm * (max_y - min_y) + min_y

In [17]:
theta0 = np.zeros((X_norm.shape[1], 1)) + 0.4

In [18]:
theta, theta_history, j_history = gradient_descent(theta0, X_norm, y_norm)

    0	 3.8723	
  100	 3.6385	
  200	 3.5909	
  300	 3.5504	
Convergence achieved.


In [19]:
y_hat = predict(X_test, theta)

In [20]:
print('Residual sum of squares: %.2f' % np.mean((y_hat - y_test) ** 2))

Residual sum of squares: 626.66


#### More information about gradient descent method is [here.](https://euanrussano.github.io/20190810linearRegressionNumpy/)